In [2]:
import numpy as np
import tensorflow as tf
import gym
from gym import spaces

class CustomFrozenLakeEnv(gym.Env):
    # (unchanged from previous code)
    def __init__(self, size=16, num_holes=10, num_bonus=10):
        self.size = size
        self.num_holes = num_holes
        self.num_bonus = num_bonus
        self.grid = np.zeros((size, size), dtype=np.int8)
        self.start_pos = (0, 0)
        self.goal_pos = (size - 1, size - 1)

        hole_positions = [(np.random.randint(size), np.random.randint(size)) for _ in range(num_holes)]
        for pos in hole_positions:
            self.grid[pos] = 1  # Mark as hole

        bonus_positions = [(np.random.randint(size), np.random.randint(size)) for _ in range(num_bonus)]
        for pos in bonus_positions:
            self.grid[pos] = 10  # Mark as bonus reward

        self.action_space = spaces.Discrete(4)  # Up, Down, Left, Right
        self.observation_space = spaces.Discrete(size * size)

        self.agent_pos = self.start_pos
        self.done = False

    def reset(self):
        self.agent_pos = self.start_pos
        self.done = False
        return self.get_state()

    def get_state(self):
        return self.agent_pos[0] * self.size + self.agent_pos[1]

    def step(self, action):
        if self.done:
            raise ValueError("Episode is done. Call reset() to start a new episode.")

        row, col = self.agent_pos

        if action == 0:  # Up
            row = max(0, row - 1)
        elif action == 1:  # Down
            row = min(self.size - 1, row + 1)
        elif action == 2:  # Left
            col = max(0, col - 1)
        elif action == 3:  # Right
            col = min(self.size - 1, col + 1)

        self.agent_pos = (row, col)

        if self.grid[self.agent_pos] == 1:  # Agent fell into a hole
            reward = -1
            self.done = True
        elif self.grid[self.agent_pos] == 10:  # Agent found a bonus reward
            reward = 10
            self.grid[self.agent_pos] = 0  # Remove bonus reward from the grid
        elif self.agent_pos == self.goal_pos:  # Agent reached the goal
            reward = 1
            self.done = True
        else:
            reward = 0

        return self.get_state(), reward, self.done, {}

    def render(self):
        render_grid = np.zeros_like(self.grid, dtype=str)
        render_grid[self.grid == 1] = 'H'
        render_grid[self.grid == 10] = '+'
        render_grid[self.start_pos] = 'S'
        render_grid[self.goal_pos] = 'G'
        render_grid[self.agent_pos] = 'A'

        print(render_grid)

class QNetwork(tf.keras.Model):
    def __init__(self, num_actions):
        super(QNetwork, self).__init__()
        self.layer1 = tf.keras.layers.Dense(64, activation='relu')
        self.layer2 = tf.keras.layers.Dense(64, activation='relu')
        self.output_layer = tf.keras.layers.Dense(num_actions, activation='linear')

    def call(self, state):
        x = self.layer1(state)
        x = self.layer2(x)
        return self.output_layer(x)

num_actions = 4
q_network = QNetwork(num_actions)

learning_rate = 0.001
gamma = 0.99
optimizer = tf.optimizers.Adam(learning_rate)
huber_loss = tf.keras.losses.Huber()

num_episodes = 1000
epsilon = 0.1

env = CustomFrozenLakeEnv(size=16, num_holes=10, num_bonus=10)

max_reward = float('-inf')
best_path = None

for episode in range(num_episodes):
    state = env.reset()
    state = np.reshape(state, [1, 1]).astype(np.float32)

    total_reward = 0
    episode_path = [env.agent_pos]  # Store the initial position

    with tf.GradientTape() as tape:
        for t in range(1000):  # Maximum number of steps per episode
            if np.random.rand() < epsilon:
                action = np.random.randint(env.action_space.n)
            else:
                q_values = q_network(state)
                action = tf.argmax(q_values, axis=1).numpy()[0]

            next_state, reward, done, _ = env.step(action)
            next_state = np.reshape(next_state, [1, 1]).astype(np.float32)

            episode_path.append(env.agent_pos)

            q_values_next = q_network(next_state)
            target = reward + gamma * tf.reduce_max(q_values_next, axis=1)
            loss = huber_loss(target, q_network(state))

            total_reward += reward

            if done:
                break

            state = next_state

    grads = tape.gradient(loss, q_network.trainable_variables)
    optimizer.apply_gradients(zip(grads, q_network.trainable_variables))

    print("Episode {}: Total Reward: {}".format(episode, total_reward))

    if total_reward > max_reward:
        max_reward = total_reward
        best_path = episode_path.copy()

    if done:
        if reward == 1:
            print("Game completed successfully! Agent reached the goal.")
        else:
            print("Agent fell into a hole. Game over.")
            print("Path Traversed:", episode_path)
            print("Maximum Reward Achieved:", max_reward)
            print("Episodes Trained For:", episode + 1)
            break

Episode 0: Total Reward: 20
Episode 1: Total Reward: -1
Agent fell into a hole. Game over.
Path Traversed: [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 1), (1, 0), (1, 1), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 1), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 